In [ ]:
%matplotlib widget

# Powder diffraction data reduction

This notebook will guide you through the data reduction for the powder diffraction experiment that you simulated with McStas yesterday.

The following is a basic outline of what this notebook will cover:

- Loading the NeXus files that contain the data
- Inspect/visualize the data contents
- How to convert the raw event `time-of-arrival` coordinate to something more useful ($\lambda$, $d$, ...)
- Improve the estimate of neutron `time-of-flight`
- Save the reduced data to disk, in a state that is ready for analysis/fitting.

In [ ]:
import numpy as np
import scipp as sc
import plopp as pp
import powder_utils as utils

## Process the run with a reference sample: Si

### Load the NeXus file data

By default, we will use the data we generated in yesterday's simulation:

In [ ]:
folder = "../3-mcstas/output_sample_Si"

⚠️ **If you did not complete the Si simulation yesterday**,
you can use some pre-prepared data by uncommenting and running the cell below:

In [ ]:
# folder = utils.fetch_data("3-mcstas/output_sample_Si")

We now load the data:

In [ ]:
sample_si = utils.load_powder(folder)

The first way to inspect the data is to view the HTML representation of the loaded object.

Try to explore what is inside the data, and familiarize yourself with the different sections (`Dimensions`, `Coordinates`, `Data`).

In [ ]:
sample_si

### Visualize the data

Here is a 2D visualization of the neutron counts, histogrammed along the `x` and `toa`dimensions:

In [ ]:
sample_si.hist(x=200, toa=200).plot(norm="log", vmin=1.0e-2)

We can also visualize the events in 3D, which show the shape of the detector panels:

In [ ]:
pp.scatter3d(sample_si[::20], pos="position", cbar=True, vmin=0.01, pixel_size=0.07)

### Coordinate transformations

The first step of this data reduction workflow is to convert the raw event coordinates (position, time-of-flight) to something physically meaningful such as wavelength ($\lambda$) or d-spacing ($d$).

Scipp has a dedicated method for this called `transform_coords` (see docs [here](https://scipp.github.io/scippneutron/user-guide/coordinate-transformations.html)).

We begin with a standard graph which describes how to compute e.g. the wavelength from the other coordinates in the raw data.

In [ ]:
from scippneutron.conversion.graph.beamline import beamline
from scippneutron.conversion.graph.tof import kinematic

graph = {**beamline(scatter=True), **kinematic("tof")}


def compute_tof(toa, time_origin):
    return toa - time_origin


graph["tof"] = compute_tof
sc.show_graph(graph, simplified=True)

To compute the wavelength of all the events, we simply call `transform_coords` on our loaded data,
requesting the name of the coordinate we want in the output (`"wavelength"`),
as well as providing it the graph to be used to compute it (i.e. the one we defined just above).

This yields

In [ ]:
si_wav = sample_si.transform_coords(["wavelength", "two_theta"], graph=graph)
si_wav

The result has a `wavelength` coordinate. We can also plot the result:

In [ ]:
si_wav.hist(two_theta=128, wavelength=200).plot(norm="log", vmin=1.0e-3)

### Exercise 1: convert raw data to d-spacing

Instead of wavelength as in the example above,
the task is now to convert the raw data to interplanar lattice spacing $d$.

The transformation graph is missing the computation for $d$ so you will have to add it in yourself.
As a reminder, $d$ is computed as follows

$$d = \frac{\lambda}{2 \sin \theta}$$

You have to:

- create a function that computes $d$
- add it to the graph with name "dspacing"
- call `transform_coords` using the new graph

Note that the graph already contains the necessary components to compute the scattering angle $2 \theta$ (called `two_theta` in code).

**Solution:**

In [ ]:
# Insert your solution:


### Histogram the data in d-spacing

The final step in processing the sample run is to histogram the data into $d$ bins.

In [ ]:
dbins = sc.linspace("dspacing", 0.7, 2.5, 901, unit="angstrom")
ntheta = 128

si_dspacing_hist = si_dspacing.hist(two_theta=ntheta, dspacing=dbins)
si_dspacing_hist.plot(norm="log", vmin=1.0e-3)

In [ ]:
si_dspacing_hist.sum("two_theta").plot()

## Exercise 2: Correct the time-of-flight

### Time-of-flight at a long-pulsed source

The time-of-flight of a neutron is typically not known from experimental measurements.
The only two parameters that we are able to record are usually the time at which the pulse was emitted (named `time_origin` in our dataset),
and the timestamp when a neutron was detected by a detector pixel (named `toa`, standing for time-of-arrival).

The difference between these two times is commonly used to compute an approximate time-of-flight.
This approximation performs well at neutron sources with a short pulse,
where it is satisfactory to assume that all neutrons left the source at the same time.

At a long-pulsed source (lasting a few milliseconds) such as ESS, with high-precision instruments,
this approximation is no longer valid.

This is in fact the reason why in the $2\theta$ vs d-spacing plot, the bright lines appear curved while they should in reality be vertical.

In the following exercise, we will add a correction to our time-of-flight computation to improve our results.

### Finding a new time-distance origin

One way to look at the problem is to visualize the paths of each neutron that made it to the detector,
on a time-distance diagram.

The neutrons begin at the source (bottom left), travel along the guide, go through the chopper,
and finally arrive at the detectors (top).

In [ ]:
utils.time_distance_diagram(si_wav)

As the figure above illustrates,
the current time-of-flight computation (`tof = toa - time_origin`) assumes that all neutrons were born at $t = 0$ (black dot).

This is clearly not the case from looking at the neutron tracks.

#### Exercise: find a new origin

- Inspect the time-distance diagram and try to find a point which would be a better representation of a common origin for all tracks.
- Set the origin as coordinates on the original data (**hint:** you will need to update the `source_position` and `time_origin` coordinates).
- Compute wavelength/d-spacing again.

**Solution:**

In [ ]:
# Insert your solution:


In [ ]:
better_dspacing.hist(two_theta=ntheta, dspacing=dbins).plot(norm="log", vmin=1.0e-3)

In [ ]:
pp.plot(
    {
        "original": si_dspacing_hist.sum("two_theta"),
        "improved": better_dspacing.hist(two_theta=ntheta, dspacing=dbins).sum(
            "two_theta"
        ),
    }
)

## Exercise 3: Process the Vanadium run

We now need to repeat the process (load data, correct time-of-flight, convert to d-spacing) for the Vanadium run.

In [ ]:
folder = "../3-mcstas/output_sample_vanadium"

⚠️ **If you did not complete the Vanadium simulation yesterday**,
you can use some pre-prepared data by uncommenting and running the cell below:

In [ ]:
# folder = utils.fetch_data("3-mcstas/output_sample_vanadium")

In [ ]:
sample_van = utils.load_powder(folder)

In [ ]:
# Insert your solution:


In [ ]:
van_dspacing.hist(two_theta=ntheta, dspacing=dbins).plot(norm="log", vmin=1.0e-3)

## Normalize by Vanadium

The next step in our data reduction is to normalize our sample signal by the Vanadium counts.

Vanadium is an almost perfect incoherent scatterer (scatters equally in all directions)
and the signal from that run can be used to correct for difference in detector exposure and sensitivity.

Performing accurate normalization is a complex subject,
but in our simple idealised set-up,
it basically boils down to a simple division operation.

We first histogram the sample and vanadium data using the same binning for both:

In [ ]:
num = better_dspacing.hist(two_theta=ntheta, dspacing=dbins)
den = van_dspacing.hist(two_theta=num.coords["two_theta"], dspacing=dbins)

For best results, it is common practice to smooth the Vanadium signal before using it for normalization,
to remove any noise that mostly comes from collection statistics rather than detector efficiency.

We thus smooth the Vandium histogram using a simple Gaussian filter.
We will normalize the signal as a 1D histogram, so the data is first summed over the `two_theta` dimension.
In Scipp, because of the physical units,
we can define a physical kernel width as opposed to having to determine how many neighbours are necessary for good smoothing.

In [ ]:
# Smooth the Vanadium run
from scipp.scipy.ndimage import gaussian_filter

smooth_vana = gaussian_filter(
    sc.values(den.sum("two_theta")), sigma=sc.scalar(0.03, unit="angstrom")
)

pp.plot(
    {"orig": den.sum("two_theta"), "smooth": smooth_vana}, title="Vanadium - smoothed"
)

### Exercise 4: perform the normalization

Finally, we divide the sample histogram `num` (summed over `two_theta`) by the smoothed Vanadium.

In [ ]:
# Insert your solution:


In [ ]:
normed.plot()

You can compare to the plot above (before normalization) and see that the relative intensities of the Si peaks have changed.

## Save to disk

We now need to save our reduced normalized data to a file,
that will then be used by the analysis software in the next chapter.

For this, we use a small helper function:

In [ ]:
utils.save_xye(
    "reduced_Si.xye",
    event_data=better_dspacing,  # Needed for saving metadata required for analysis
    normalized_histogram=normed,
)

## Exercise 5: Process and normalize main sample: LBCO

Repeat the process for the LBCO sample:

- load data
- correct time-of-flight
- convert to d-spacing
- normalize by Vanadium
- save to disk

In [ ]:
folder = "../3-mcstas/output_sample_LBCO"

⚠️ **If you did not complete the Si simulation yesterday**,
you can use some pre-prepared data by uncommenting and running the cell below:

In [ ]:
# folder = utils.fetch_data("3-mcstas/output_sample_LBCO")

In [ ]:
# Insert your solution:


In [ ]:
# Insert your solution:


## Sciline workflow

<div class="alert alert-warning">

**Pause here if you have not yet been introduced to Sciline!**

</div>

Below we build a re-usable Sciline workflow for reducing the powder diffraction data.
We wrap the operations that were carried out throughout this notebook into function that have the correct type-annotations,
building a pipeline of connected steps.

The custom types have been created for you in the `powder_utils` module, and are imported from there.

Take some time to read through the code and understand the different parts.
Ask questions if anything is unclear.

In [ ]:
import sciline as sl
from powder_utils import *


def load(folder: Foldername[RunType]) -> RawData[RunType]:
    """Load raw data from file"""
    return RawData[RunType](utils.load_powder(folder))


def correct_beamline_params(
    data: RawData[RunType], source_position: SourcePosition, time_origin: TimeOrigin
) -> CorrectedData[RunType]:
    """Correct the source position and time origin for a better time-of-flight estimate"""
    corrected = data.copy(deep=False)
    corrected.coords["source_position"] = source_position
    corrected.coords["time_origin"] = time_origin
    return CorrectedData[RunType](corrected)


def to_dspacing(
    data: CorrectedData[RunType], graph: CoordTransformGraph
) -> DspacingData[RunType]:
    """Compute dspacing for events"""
    return DspacingData[RunType](data.transform_coords("dspacing", graph=graph))


def to_histogram(
    data: DspacingData[RunType],
    two_theta_bins: TwoThetaBins,
    dspacing_bins: DspacingBins,
) -> HistogramTwothetaDspacing[RunType]:
    """Histogram event data using two_theta and dspacing binning"""
    return HistogramTwothetaDspacing[RunType](
        data.hist(two_theta=two_theta_bins, dspacing=dspacing_bins)
    )


def smooth_vanadium(
    van: HistogramTwothetaDspacing[VanadiumRun], sigma: GaussianSmoothingSigma
) -> SmoothedVanadium:
    """Smooth the Vanadium data using a Gaussian kernel"""
    return SmoothedVanadium(
        gaussian_filter(sc.values(van.sum("two_theta")), sigma=sigma)
    )


def sum_over_two_theta(
    sample: HistogramTwothetaDspacing[SampleRun],
) -> SampleDspacing:
    """Sum the sample data over the two_theta dimension so we are left with dspacing"""
    return SampleDspacing(sample.sum("two_theta"))


def normalize(sample: SampleDspacing, van: SmoothedVanadium) -> NormalizedDspacing:
    """Normalize sample by vanadium"""
    return NormalizedDspacing(sample / van)


wf = sl.Pipeline(
    (
        load,
        correct_beamline_params,
        to_dspacing,
        to_histogram,
        smooth_vanadium,
        sum_over_two_theta,
        normalize,
    ),
    constraints={RunType: [SampleRun, VanadiumRun]},
)

wf.visualize()

### Exercise 6: Set the workflow parameters

In the visualization above, the red boxes indicate missing values;
parameters required by the computation that have yet to be set on the workflow.

Using the `wf[TYPE] = VALUE` syntax, set the missing parameters on the workflow.

Once you have done so, use `wf.compute(NormalizedDspacing)` to compute the final normalized result for the Si and LBCO samples,
remembering to update the `Foldername[SampleRun]` for each case.

**Solution:**

In [ ]:
# Insert your solution:


### Exercise 7 (optional): Modify the dspacing binning

Change the number of bins used in the dspacing dimension on the workflow,
and recompute the result.

**Hint:** use `wf[DspacingBins] = ...`

**Solution:**

In [ ]:
# Insert your solution:


### Exercise 8: Debugging detector banks

The detector team suspects that one of the two cylindrical banks is faulty (recording too many counts),
and you have to help them find out which one (if any!).

- Identify an intermediate result in the workflow graph that contains the necessary information.
- Separate the events between left and right detector banks (hint: you can either use masks or smart indexing: `da[da.coords['x'] < x0]`)
- Histogram both banks using the dspacing bins
- Plot both histograms on the same axes (hint: send a dict to the plotting function `pp.plot({"left": left_hist, "right": right_hist})`)

**Solution:**

In [ ]:
# Insert your solution:
